# Decorator

## 기존의 클래스나 함수를 수정하지 않고 기능을 덧붙일 수 있는 역할

In [3]:
def deco(func):

    def wrapper():
        print ("before")
        ret = func()
        print ("after")
        return ret

    return wrapper

#@deco
def base():
    print ("base function")



print ("=== Run decorator ===")
#base()
argument =  base
f = deco(argument)
f()

=== Run decorator ===
before
base function
after


## 다중 Decorator

In [6]:
import time
import datetime

def measure_run_time(func):

    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()

        print ("'%s' function running time : %s" % (func.__name__, end - start))
        return result

    return wrapper

def parameter_logger(func):

    def wrapper(*args, **kwargs):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        print ("[%s] args : %s, kwargs : %s" % (timestamp, args, kwargs))
        return func(*args, **kwargs)

    return wrapper

@measure_run_time
@parameter_logger
def worker(delay_time):
    time.sleep(delay_time)


worker(5)

[2019-01-22 15:01] args : (5,), kwargs : {}
'wrapper' function running time : 5.00518012046814


 - 다음과 같이 풀어보면 measure_run_time의 argument가 worker함수가 아닌 parameter_logger의반환값 즉, wrapper함수임을 알 수 있다.

In [19]:
import time
import datetime

def measure_run_time(func):

    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()

        print ("'%s' function running time : %s" % (func.__name__, end - start))
        return result

    return wrapper

def parameter_logger(func):

    def wrapper(*args, **kwargs):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        print ("[%s] args : %s, kwargs : %s" % (timestamp, args, kwargs))
        return func(*args, **kwargs)

    return wrapper

def worker(delay_time):
    time.sleep(delay_time)


argument = worker
f1 = parameter_logger(argument)
f2 = measure_run_time(f1)

f2(1)
print(f1.__closure__[0].cell_contents)

[2019-01-22 15:24] args : (1,), kwargs : {}
'wrapper' function running time : 1.0013411045074463
<function worker at 0x7f96300a12f0>


## @wraps
- wrap를 써주면 wrapper함수가 아닌 메인함수의 속성을 알 수 있어 디버깅이 편하다.

In [12]:
import time
import datetime
from functools import wraps

def measure_run_time(func):

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()

        print ("'%s' function running time : %s" % (func.__name__, end - start))
        return result

    return wrapper

def parameter_logger(func):

    @wraps(func)
    def wrapper(*args, **kwargs):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        print ("[%s] args : %s, kwargs : %s" % (timestamp, args, kwargs))
        return func(*args, **kwargs)

    return wrapper

@measure_run_time
@parameter_logger
def worker(delay_time):
    time.sleep(delay_time)


worker(5)

[2019-01-22 15:21] args : (5,), kwargs : {}
'worker' function running time : 5.005084037780762


## Class Decorator

In [28]:
import time
from functools import update_wrapper
from functools import wraps

class MeasureRuntime:

    def __init__(self, f):
        self.func = f
        update_wrapper(self, self.func)

    def __call__(self, *args, **kwargs):
        start = time.time()
        result = self.func(*args, **kwargs)
        end = time.time()
        print ("'%s' function running time : %s" % (self.func.__name__, end - start))
        return result


@MeasureRuntime
def worker(delay_time):
    time.sleep(delay_time)

#f = MeasureRuntime(worker)
#f(5)
worker(1)

'worker' function running time : 1.001089334487915


# Iterable & Iterator

- 외부함수 -> __init__ 함수
- 내부함수 -> __call__ 함수

In [29]:
import time
from functools import wraps

class MeasureRuntime:

    def __init__(self, active_state):
        self.measure_active = active_state

    def __call__(self, func):

        @wraps(func)
        def wrapper(*args, **kwargs):
            if self.measure_active is False:
                return func(*args, **kwargs)

            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print ("'%s' function running time : %s" % (func.__name__, end - start))
            return result

        return wrapper


@MeasureRuntime(True)
def active_worker(delay_time):
    time.sleep(delay_time)

@MeasureRuntime(False)
def non_active_worker(delay_time):
    time.sleep(delay_time)

    
active_worker(5)
non_active_worker(5)

'active_worker' function running time : 5.005074977874756


- iterable : 한번에 하나씩 반환할 수 있는 개체
- iterator : 한번에 하나씩만 반환할 수 있는 개체

In [30]:
nums = [1,2,3,4,5]

In [31]:
next(nums)

TypeError: 'list' object is not an iterator

In [62]:
iter_nums = iter(nums)

In [63]:
print(next(iter_nums))

1


- Iterator의 특징은 이미 반환한 상태와 어디서 부터 반환해야 할지의 상태를 가지고 있다는 것이다.

# Generator
- Genrator iterator을 반환하는 함수이다.
- Iterator는 값을 모두 연산한 뒤 하나씩 값을 반환하고, Generator iterator은 값을 반환할 때 연산을 수행한다.
- yield 구문이 있어야 한다.

## yield
- Generator에서 값을 반환하거나 값을 입력받는 기능
- Generator에서 yield를 만나면 현재 상태 그대로 멈춘다.

In [66]:
def gen():
    yield 1
    yield 2
    yield 3

def normal():
    return 1
    return 2
    return 3

def main():
    print ("=== print gen function ===")
    print (gen())

    print ("=== print normal function===")
    print (normal())

    print ("=== print gen function in loop ===")
    for g in gen():
        print (g)

    print ("=== print normal function in loop ===")
    for n in normal():
        print (n)



main()

=== print gen function ===
<generator object gen at 0x7f96300397d8>
=== print normal function===
1
=== print gen function in loop ===
1
2
3
=== print normal function in loop ===


TypeError: 'int' object is not iterable

In [68]:
def gen():
    value = 1
    while True:
        value = yield value

def main():
    print ("=== print gen function ===")
    g = gen()

    print (next(g))
    print (g.send(2))
    print (g.send(10))
    print (g.send(5))
    print (next(g))


if __name__ == "__main__":
    main()

=== print gen function ===
1
2
10
5
None


## Stateful generator

In [69]:
def gen(items):
    count = 0

    for item in items:
        if count == 10:
            return -1

        count += 1
        yield item


if __name__ == "__main__":
    print ("=== print gen ===")
    for i in gen(range(15)):
        print (i)

=== print gen ===
0
1
2
3
4
5
6
7
8
9


## Generator expression

In [101]:
SAMPLE_LIST = [1, 2, 3, 4, 5]

def generate_sample_list():
    result = (x*x for x in SAMPLE_LIST)
    print (result)
    return result

def generate_list_by_range():
    result = (i for i in range(10))
    print (result)
    return result

def print_generator(items):
    for item in items:
        print (item)

def main():
    print ("=== print list ===")
    print_generator(generate_sample_list())
    print_generator(generate_list_by_range())


if __name__ == "__main__":
    main()

=== print list ===
<generator object generate_sample_list.<locals>.<genexpr> at 0x7f9621f57f10>
1
4
9
16
25
<generator object generate_list_by_range.<locals>.<genexpr> at 0x7f9621f57f10>
0
1
2
3
4
5
6
7
8
9


## Lazy evaluation

In [104]:
import time

def wait_return(num):
    print ("sleep")
    time.sleep(0.5)
    return num

def print_items(items):
    for i in items:
        print (i)

def main():
    print ("=== print list comprehension ===")
    iterator_list = [wait_return(i) for i in range(10)]
    print("check", iterator_list)
    print_items(iterator_list)

    print ("=== print generator expression ===")
    iterator_list = (wait_return(i) for i in range(10))
    print_items(iterator_list)


if __name__ == "__main__":
    main()

=== print list comprehension ===
sleep
sleep
sleep
sleep
sleep
sleep
sleep
sleep
sleep
sleep
check [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
1
2
3
4
5
6
7
8
9
=== print generator expression ===
sleep
0
sleep
1
sleep
2
sleep
3
sleep
4
sleep
5
sleep
6
sleep
7
sleep
8
sleep
9
